# Python Flair Basics

**(C) 2018-2020 by [Damir Cavar](http://damir.cavar.me/)**

**Version:** 0.3, February 2020

**Download:** This and various other Jupyter notebooks are available from my [GitHub repo](https://github.com/dcavar/python-tutorial-for-ipython).

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

This material was used in my Advanced Topics in AI class, introduction to Deep Learning environments in Spring 2019 at Indiana University.

Two types of central objects:

- Sentence
- Token

A Sentence holds a textual sentence and is essentially a list of Token objects.

For creating a Sentence object we first import the Sentence class from the *flair.data* module:

In [48]:
from flair.data import Sentence

We can now define a sentence:

In [49]:
sentence = Sentence('The grass is green .')
print(sentence)

Sentence: "The grass is green ."   [− Tokens: 5]


We can access the tokens of a sentence via their token id or with their index:

In [50]:
print(sentence.get_token(4))
print(sentence[3])

Token: 4 green
Token: 4 green


We can also iterate over all tokens in a sentence:

In [51]:
for token in sentence:
    print(token)

Token: 1 The
Token: 2 grass
Token: 3 is
Token: 4 green
Token: 5 .


## Tokenization

There is a simple tokenizer included in the library using the lightweight segtok library to tokenize your text for such a Sentence definition. In the Sentence constructor use the flag *use_tokenize* to tokenize the input string before instantiating a Sentence object:

In [52]:
sentence = Sentence('The grass is green.', use_tokenizer=True)
print(sentence)

Sentence: "The grass is green ."   [− Tokens: 5]


## Tags on Tokens

A Token has fields for linguistic annotation:

- lemma
- part-of-speech tag
- named entity tag

We can add a tag by specifying the tag type and the tag value.

We will adding an NER tag of type 'color' to the word 'green'. This means that we've tagged this word as an entity of type color:

In [53]:
sentence[3].add_tag('ner', 'color')
print(sentence.to_tagged_string())

The grass is green <color> .


Each tag is of class Label. An associated score indicates confidence:

In [54]:
from flair.data import Label

tag: Label = sentence[3].get_tag('ner')
print(f'"{sentence[3]}" is tagged as "{tag.value}" with confidence score "{tag.score}"')

"Token: 4 green" is tagged as "color" with confidence score "1.0"


The manually added color tag has a score of 1.0. A tag predicted by a sequence labeler will have a score value that indicates the classifier confidence.

A Sentence can have one or multiple labels that can for example be used in text classification tasks. For instance, the example below shows how we add the label 'sports' to a sentence, thereby labeling it as belonging to the sports category.

In [55]:
sentence = Sentence('France is the current world cup winner.')
sentence.add_label('topic', 'sports')
sentence.add_label('language', 'English')
print(sentence)
# sentence.add_labels(1, ['sports', 'world cup'])
sentence = Sentence('France is the current world cup winner.').add_label('topic', 'sports')
print(sentence)

Sentence: "France is the current world cup winner ."   [− Tokens: 8  − Sentence-Labels: {'topic': [sports (1.0)], 'language': [English (1.0)]}]
Sentence: "France is the current world cup winner ."   [− Tokens: 8  − Sentence-Labels: {'topic': [sports (1.0)]}]


Labels are also of the Label class. So, you can print a sentence's labels like this:

In [10]:
sentence = Sentence('France is the current world cup winner.').add_label('sports', 'world cup')

print(sentence)
for label in sentence.labels:
    print(label)

Sentence: "France is the current world cup winner ."   [− Tokens: 8  − Sentence-Labels: {'sports': [world cup (1.0)]}]
world cup (1.0)


## Tagging Text

### Using Pre-Trained Sequence Tagging Models

Flair has numerous pre-trained models. For the named entity recognition (NER) task there is a model that was trained on the English CoNLL-03 task and can recognize 4 different entity types. Import it using:

In [57]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load('ner')

2020-11-04 17:54:35,671 loading file /home/damir/.flair/models/en-ner-conll03-v0.4.pt


We use the predict() method of the tagger on a sentence to add predicted tags to the tokens in the sentence:

In [58]:
sentence = Sentence('Washington went to New York .')

tagger.predict(sentence)

print(sentence.to_tagged_string())

Washington <S-PER> went to New <B-LOC> York <E-LOC> .


Getting annotated spans for multi-word expressions can be achieved using the following command:

In [59]:
for entity in sentence.get_spans('ner'):
    print(entity)

Span [1]: "Washington"   [− Labels: PER (0.999)]
Span [4,5]: "New York"   [− Labels: LOC (0.9996)]


Which indicates that "George Washington" is a person (PER) and "Washington" is a location (LOC). Each such Span has a text, a tag value, its position in the sentence and "score" that indicates how confident the tagger is that the prediction is correct. You can also get additional information, such as the position offsets of each entity in the sentence by calling:

In [60]:
print(sentence.to_dict(tag_type='ner'))

{'text': 'Washington went to New York .', 'labels': [], 'entities': [{'text': 'Washington', 'start_pos': 0, 'end_pos': 10, 'labels': [PER (0.999)]}, {'text': 'New York', 'start_pos': 19, 'end_pos': 27, 'labels': [LOC (0.9996)]}]}


In [61]:
print(sentence.to_dict())

{'text': 'Washington went to New York .', 'labels': [], 'entities': []}


Flair contains various sequence tagger models. You choose which pre-trained model you load by passing the appropriate string to the load() method of the SequenceTagger class. Currently, the following pre-trained models are provided:

As indicated in the list above, we also provide pre-trained models for languages other than English. Currently, we support German, French, and Dutch other languages are forthcoming. To tag a German sentence, just load the appropriate model:

In [63]:
tagger = SequenceTagger.load('de-ner')

sentence = Sentence('George Washington ging nach Washington .')

tagger.predict(sentence)

print(sentence.to_tagged_string())

2020-11-04 17:56:38,303 loading file /home/damir/.flair/models/de-ner-conll03-v0.4.pt
George <B-PER> Washington <E-PER> ging nach Washington <S-LOC> .


Flair offers access to multi-lingual models for multi-lingual text.

In [64]:
tagger = SequenceTagger.load('pos-multi')

sentence = Sentence('George Washington lebte in Washington . Dort kaufte er a horse .')

tagger.predict(sentence)

print(sentence.to_tagged_string())

2020-11-04 17:57:10,840 loading file /home/damir/.flair/models/pos-multi-v0.1.pt
George <PROPN> Washington <PROPN> lebte <VERB> in <ADP> Washington <PROPN> . <PUNCT> Dort <PROPN> kaufte <VERB> er <PRON> a <DET> horse <NOUN> . <PUNCT>


## Semantic Frames

For English, Flair provides a pre-trained model that detects semantic frames in text, trained using Propbank 3.0 frames. This provides some sort of word sense disambiguation for frame evoking words.

In [65]:
tagger = SequenceTagger.load('frame')

sentence_1 = Sentence('George returned to Berlin to return his hat .')
sentence_2 = Sentence('He had a look at different hats .')

tagger.predict(sentence_1)
tagger.predict(sentence_2)

print(sentence_1.to_tagged_string())
print(sentence_2.to_tagged_string())

2020-11-04 17:58:12,060 loading file /home/damir/.flair/models/en-frame-ontonotes-v0.4.pt
George returned <return.01> to Berlin to return <return.02> his hat .
He had <have.03> a look <look.01> at different hats .


The frame detector makes a distinction in sentence 1 between two different meanings of the word 'return'. 'return.01' means returning to a location, while 'return.02' means giving something back.

Similarly, in sentence 2 the frame detector finds a light verb construction in which 'have' is the light verb and 'look' is a frame evoking word.

## Sentence Tagging

To tag an entire text corpus, one needs to split the corpus into sentences and pass a list of Sentence objects to the .predict() method.

In [66]:
text = "This is a sentence. John read a book. This is another sentence. I love Berlin."

from segtok.segmenter import split_single

sentences = [Sentence(sent, use_tokenizer=True) for sent in split_single(text)]

tagger: SequenceTagger = SequenceTagger.load('ner')
tagger.predict(sentences)
for i in sentences:
    print(i.to_tagged_string())

2020-11-04 17:59:30,738 loading file /home/damir/.flair/models/en-ner-conll03-v0.4.pt
This is a sentence .
John <S-PER> read a book .
This is another sentence .
I love Berlin <S-LOC> .


Using the mini_batch_size parameter of the .predict() method, you can set the size of mini batches passed to the tagger. Depending on your resources, you might want to play around with this parameter to optimize speed.

## Pre-Trained Text Classification Models

Flair provides a pre-trained model for detecting positive or negative comments. It was trained on the IMDB dataset and it can recognize positive and negative sentiment in English text. The IMDB data set can be downloaded from the [linked site](http://ai.stanford.edu/~amaas/data/sentiment/).

In [67]:
from flair.models import TextClassifier

classifier = TextClassifier.load('en-sentiment')

2020-11-04 17:59:56,044 loading file /home/damir/.flair/models/sentiment-en-mix-distillbert_3.1.pt


Widget Javascript not detected.  It may not be installed or enabled properly.


We call the predict() method of the classifier on a sentence. This will add the predicted label to the sentence:

In [68]:
sentence = Sentence('This film hurts. It is so bad that I am confused.')

classifier.predict(sentence)

print(sentence.labels)

[NEGATIVE (1.0)]


In [69]:
sentence = Sentence('This film is fantastic. I love it.')

classifier.predict(sentence)

print(sentence.labels)

[POSITIVE (0.9977)]


Flair has a pre-trained German and English model.

## Using Word Embeddings

Flair provides a set of classes with which we can embed the words in sentences in various ways.

All word embedding classes inherit from the TokenEmbeddings class and implement the embed() method which we need to call to embed our text. This means that for most users of Flair, the complexity of different embeddings remains hidden behind this interface. Simply instantiate the embedding class we require and call embed() to embed our text.

All embeddings produced with Flair's methods are Pytorch vectors, so they can be immediately used for training and fine-tuning.

Classic word embeddings are static and word-level, meaning that each distinct word gets exactly one pre-computed embedding. Most embeddings fall under this class, including the popular GloVe or Komninos embeddings.

We instantiate the WordEmbeddings class and pass a string identifier of the embedding we wish to load. If we want to use GloVe embeddings, we pass the string 'glove' to the constructor:

In [71]:
from flair.embeddings import WordEmbeddings

glove_embedding = WordEmbeddings('glove')

We create an example sentence and call the embedding's embed() method. We can also pass a list of sentences to this method since some embedding types make use of batching to increase speed.

In [73]:
sentence = Sentence('The grass is green .')

glove_embedding.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)
print(sentence.embedding)

Token: 1 The
tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  

GloVe embeddings are Pytorch vectors of dimensionality 100.

We choose which pre-trained embeddings we want to load by passing the appropriate id string to the constructor of the WordEmbeddings class. We would use the two-letter language code to init an embedding, so 'en' for English and 'de' for German and so on. By default, this will initialize FastText embeddings trained over Wikipedia. We can also always use FastText embeddings over Web crawls, by instantiating with '-crawl'. The 'de-crawl' option would use embeddings trained over German web crawls.

For English, Flair provides a few more options. We can choose between instantiating 'en-glove', 'en-extvec' and so on.

If we want to load German FastText embeddings, instantiate as follows:

In [74]:
german_embedding = WordEmbeddings('de')

If we want to load German FastText embeddings trained over crawls, we instantiate as follows:

In [75]:
german_embedding = WordEmbeddings('de-crawl')

If the models are not locally available, Flair will automatically download them and install them into the local user cache.

It is recommended to use the FastText embeddings, or GloVe if we want a smaller model.

If we want to use any other embeddings (not listed in the list above), we can load those by calling:

In [ ]:
custom_embedding = WordEmbeddings('path/to/your/custom/embeddings.gensim')

If we want to load custom embeddings, we need to make sure that the custom embeddings are correctly formatted to gensim.

We can, for example, convert FastText embeddings to gensim using the following code snippet:

In [ ]:
import gensim

word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/path/to/fasttext/embeddings.txt', binary=False)
word_vectors.save('/path/to/converted')

## Character Embeddings

Some embeddings - such as character-features - are not pre-trained but rather trained on the downstream task. Normally this requires an implementation of a hierarchical embedding architecture.

With Flair, we don't need to worry about such things. Just choose the appropriate embedding class and the character features will then automatically train during downstream task training.

In [77]:
from flair.embeddings import CharacterEmbeddings

embedding = CharacterEmbeddings()

sentence = Sentence('The grass is green .')

embedding.embed(sentence)

[Sentence: "The grass is green ."   [− Tokens: 5]]


### Sub-Word Embeddings

Flair now also includes the byte pair embeddings calulated by @bheinzerling that segment words into subsequences. This can dramatically reduce the model size vis-a-vis using normal word embeddings at nearly the same accuracy. So, if we want to train small models try out the new BytePairEmbeddings class.

We initialize with a language code (275 languages supported), a number of 'syllables', and a number of dimensions (one of 50, 100, 200 or 300). The following initializes and uses byte pair embeddings for English:

In [78]:
from flair.embeddings import BytePairEmbeddings

embedding = BytePairEmbeddings('en')

sentence = Sentence('The grass is green .')

embedding.embed(sentence)

[Sentence: "The grass is green ."   [− Tokens: 5]]

[Sub-word embeddings](https://nlp.h-its.org/bpemb/) are interesting, since 

[BPEmb](https://nlp.h-its.org/bpemb/) is a collection of pre-trained subword embeddings in 275 languages, based on Byte-Pair Encoding (BPE) and trained on Wikipedia. Its intended use is as input for neural models in natural language processing.

- subwords allow guessing the meaning of unknown / out-of-vocabulary words. E.g., the suffix -shire in Melfordshire indicates a location.
- Byte-Pair Encoding gives a subword segmentation that is often good enough, without requiring tokenization or morphological analysis. In this case the BPE segmentation might be something like melf ord shire.
- Pre-trained byte-pair embeddings work surprisingly well, while requiring no tokenization and being much smaller than alternatives: an 11 MB BPEmb English model matches the results of the 6 GB FastText model in our evaluation.

If you are using word embeddings like word2vec or GloVe, you have probably encountered out-of-vocabulary words, i.e., words for which no embedding exists. A makeshift solution is to replace such words with an UNK token and train a generic embedding representing such unknown words.

Subword approaches try to solve the unknown word problem differently, by assuming that you can reconstruct a word's meaning from its parts. For example, the suffix -shire lets you guess that Melfordshire is probably a location, or the suffix -osis that Myxomatosis might be a sickness.

There are many ways of splitting a word into subwords. A simple method is to split into characters and then learn to transform this character sequence into a vector representation by feeding it to a convolutional neural network (CNN) or a recurrent neural network (RNN), usually a long-short term memory (LSTM). This vector representation can then be used like a word embedding.

Another, more linguistically motivated way is a morphological analysis, but this requires tools and training data which might not be available for your language and domain of interest.

Enter Byte-Pair Encoding (BPE) [Sennrich et al, 2016], an unsupervised subword segmentation method. BPE starts with a sequence of symbols, for example characters, and iteratively merges the most frequent symbol pair into a new symbol.

For example, applying BPE to English might first merge the characters h and e into a new symbol he, then t and h into th, then t and he into the, and so on.

Learning these merge operations from a large corpus (e.g. all Wikipedia articles in a given language) often yields reasonable subword segementations. For example, a BPE model trained on English Wikipedia splits melfordshire into mel, ford, and shire.

### Stacked Embeddings

Stacked embeddings are one of the most important concepts of Flair. We can use them to combine different embeddings together, for instance if we want to use both traditional embeddings together with contextual string embeddings (see below). Stacked embeddings allow us to mix and match. We find that a combination of embeddings often gives best results.

All we need to do is use the StackedEmbeddings class and instantiate it by passing a list of embeddings that we wish to combine. For instance, lets combine classic GloVe embeddings with character embeddings. This is effectively the architecture proposed in (Lample et al., 2016).

In [35]:
from flair.embeddings import WordEmbeddings, CharacterEmbeddings

glove_embedding = WordEmbeddings('glove')

character_embeddings = CharacterEmbeddings()

Now instantiate the StackedEmbeddings class and pass it a list containing these two embeddings.

In [36]:
from flair.embeddings import StackedEmbeddings

stacked_embeddings = StackedEmbeddings(
    embeddings=[glove_embedding, character_embeddings])

We use this embedding like all the other embeddings, i.e. call the embed() method over our sentences.

In [37]:
sentence = Sentence('The grass is green .')

stacked_embeddings.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-3.8194e-02, -2.4487e-01,  7.2812e-01, -3.9961e-01,  8.3172e-02,
         4.3953e-02, -3.9141e-01,  3.3440e-01, -5.7545e-01,  8.7459e-02,
         2.8787e-01, -6.7310e-02,  3.0906e-01, -2.6384e-01, -1.3231e-01,
        -2.0757e-01,  3.3395e-01, -3.3848e-01, -3.1743e-01, -4.8336e-01,
         1.4640e-01, -3.7304e-01,  3.4577e-01,  5.2041e-02,  4.4946e-01,
        -4.6971e-01,  2.6280e-02, -5.4155e-01, -1.5518e-01, -1.4107e-01,
        -3.9722e-02,  2.8277e-01,  1.4393e-01,  2.3464e-01, -3.1021e-01,
         8.6173e-02,  2.0397e-01,  5.2624e-01,  1.7164e-01, -8.2378e-02,
        -7.1787e-01, -4.1531e-01,  2.0335e-01, -1.2763e-01,  4.1367e-01,
         5.5187e-01,  5.7908e-01, -3.3477e-01, -3.6559e-01, -5.4857e-01,
        -6.2892e-02,  2.6584e-01,  3.0205e-01,  9.9775e-01, -8.0481e-01,
        -3.0243e+00,  1.2540e-02, -3.6942e-01,  2.2167e+00,  7.2201e-01,
        -2.4978e-01,  9.2136e-01,  3.4514e-02,  4.6745e-01,  1.1079e+00,
        -1.9358e-01, -7.4575e-02,  2.3

Words are now embedded using a concatenation of two different embeddings. This means that the resulting embedding vector is still a single PyTorch vector.

## Other Embeddings: BERT, ELMo, Flair

Next to standard WordEmbeddings and CharacterEmbeddings, Flair also provides classes for BERT, ELMo and Flair embeddings. These embeddings enable us to train truly state-of-the-art NLP models.

All word embedding classes inherit from the TokenEmbeddings class and implement the embed() method which we need to call to embed our text. This means that for most users of Flair, the complexity of different embeddings remains hidden behind this interface. We instantiate the embedding class we require and call embed() to embed our text.

All embeddings produced with Flair's methods are Pytorch vectors, so they can be immediately used for training and fine-tuning.

### Flair Embeddings

Contextual string embeddings are powerful embeddings that capture latent syntactic-semantic information that goes beyond standard word embeddings. Key differences are: (1) they are trained without any explicit notion of words and thus fundamentally model words as sequences of characters. And (2) they are contextualized by their surrounding text, meaning that the same word will have different embeddings depending on its contextual use.

Recent advances in language modeling using recurrent neural networks have made it viable to model language as distributions over characters. By learning to predict the next character on the basis of previous characters, such models have been shown to automatically internalize linguistic concepts such as words, sentences, subclauses and even sentiment. In Flair the internal states of a trained character language model is leveraged to produce a novel type of word embedding which the authors refer to as contextual string embeddings. The proposed embeddings have the distinct properties that they (a) are trained without any explicit notion of words and thus fundamentally model words as sequences of characters, and (b) are contextualized by their surrounding text, meaning that the same word will have different embeddings depending on its contextual use. The authors conduct a comparative evaluation against previous embeddings and find that their embeddings are highly useful for downstream tasks: across four classic sequence labeling tasks they consistently outperform the previous state-of-the-art. In particular, they significantly outperform previous work on English and German named entity recognition (NER), allowing them to report new state-of-the-art F1-scores on the CONLL03 shared task.

With Flair, we can use these embeddings simply by instantiating the appropriate embedding class, same as standard word embeddings:

In [39]:
from flair.embeddings import FlairEmbeddings

flair_embedding_forward = FlairEmbeddings('news-forward')

sentence = Sentence('The grass is green .')

flair_embedding_forward.embed(sentence)

[Sentence: "The grass is green ."   [− Tokens: 5]]

We can choose which embeddings we load by passing the appropriate string to the constructor of the FlairEmbeddings class. Currently, there are numerous contextual string embeddings provided in models (more coming). See [list](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md).

The recommendation is to combine both forward and backward Flair embeddings. Depending on the task, it is also recommended to add standard word embeddings into the mix. So, the recommendation is to use StackedEmbedding for most English tasks:

In [41]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

stacked_embeddings = StackedEmbeddings([
                                        WordEmbeddings('glove'), 
                                        FlairEmbeddings('news-forward'), 
                                        FlairEmbeddings('news-backward'),
                                       ])

We would use this embedding like all the other embeddings, i.e. call the embed() method over our sentences.

### BERT Embeddings

BERT embeddings were developed by Devlin et al. (2018) and are a different kind of powerful word embedding based on a bidirectional transformer architecture. Flair is using the implementation of huggingface. The embeddings are wrapped into our simple embedding interface, so that they can be used like any other embedding.

In [46]:
from flair.embeddings import TransformerWordEmbeddings

embedding = TransformerWordEmbeddings()

sentence = Sentence('The grass is green .')

embedding.embed(sentence)

Widget Javascript not detected.  It may not be installed or enabled properly.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [43]:
for i in sentence:
    print(i, i.embedding)

Token: 1 The tensor([-0.0021,  0.0005,  0.0469,  ..., -0.0004, -0.0393,  0.0106])
Token: 2 grass tensor([-0.0006,  0.0047,  0.0248,  ..., -0.0004, -0.0236,  0.0117])
Token: 3 is tensor([ 0.0011, -0.0032,  0.0156,  ..., -0.0061,  0.0112,  0.0100])
Token: 4 green tensor([-0.0034,  0.0003,  0.0256,  ..., -0.0026, -0.0118,  0.0455])
Token: 5 . tensor([ 0.0008,  0.0002,  0.1262,  ..., -0.0002,  0.0039,  0.0058])


We can load any of the pre-trained BERT models by providing the model string during initialization:

- 'bert-base-uncased': English; 12-layer, 768-hidden, 12-heads, 110M parameters
- 'bert-large-uncased': English; 24-layer, 1024-hidden, 16-heads, 340M parameters
- 'bert-base-cased': English; 12-layer, 768-hidden, 12-heads , 110M parameters
- 'bert-large-cased': English; 24-layer, 1024-hidden, 16-heads, 340M parameters
- 'bert-base-multilingual-cased': 104 languages; 12-layer, 768-hidden, 12-heads, 110M parameters
- 'bert-base-chinese': Chinese Simplified and Traditional; 12-layer, 768-hidden, 12-heads, 110M parameters

### ELMo Embeddings

ELMo embeddings were presented by Peters et al. in 2018. They are using a bidirectional recurrent neural network to predict the next word in a text. Flair is using the implementation of AllenNLP. As this implementation comes with a lot of sub-dependencies, which Flair authors don't want to include in Flair, you need to first install the library via `pip install allennlp` before we can use it in Flair. Using the embeddings is as simple as using any other embedding type. To specify the correct model, pick one of *small*, *medium*, *portugese*, *original*:

In [ ]:
from flair.embeddings import ELMoEmbeddings

embedding = ELMoEmbeddings(model="small")

sentence = Sentence('The grass is green .')

embedding.embed(sentence)

AllenNLP provides the following pre-trained models. To use any of the following models inside Flair simple specify the embedding id when initializing the ELMoEmbeddings.

- 'small': English; 1024-hidden, 1 layer, 14.6M parameters
- 'medium': English; 2048-hidden, 1 layer, 28.0M parameters
- 'original': English; 4096-hidden, 2 layers, 93.6M parameters
- 'pt': Portuguese
- 'pubmed': English biomedical data; more information

### BERT and Flair Combined

We can very easily mix and match Flair, ELMo, BERT and classic word embeddings. We instantiate each embedding we wish to combine and use them in a StackedEmbedding.

For instance, let's say we want to combine the multilingual Flair and BERT embeddings to train a hyper-powerful multilingual downstream task model.

First, instantiate the embeddings we wish to combine:

In [54]:
from flair.embeddings import FlairEmbeddings, BertEmbeddings

flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

bert_embedding = BertEmbeddings('bert-base-multilingual-cased')

Now we instantiate the StackedEmbeddings class and pass it a list containing these three embeddings:

In [55]:
from flair.embeddings import StackedEmbeddings

stacked_embeddings = StackedEmbeddings(
    embeddings=[flair_forward_embedding, flair_backward_embedding, bert_embedding])

We use this embedding like all the other embeddings, i.e. call the embed() method over our sentences.

In [56]:
sentence = Sentence('The grass is green .')

stacked_embeddings.embed(sentence)

for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-1.4812e-07,  4.5007e-08,  6.0273e-07,  ...,  3.8287e-01,
         4.7210e-01,  2.9850e-01])
Token: 2 grass
tensor([ 1.6254e-04,  1.8764e-07, -7.9034e-09,  ...,  8.5283e-01,
        -5.0724e-02,  3.4476e-01])
Token: 3 is
tensor([-2.4521e-04,  3.4869e-07,  5.5841e-06,  ..., -1.8283e-01,
         7.1532e-01,  5.0825e-03])
Token: 4 green
tensor([8.3005e-05, 4.7261e-08, 5.7316e-07,  ..., 1.0157e+00, 7.5358e-01,
        1.1230e-01])
Token: 5 .
tensor([-8.3244e-07,  1.6451e-07, -1.7201e-08,  ..., -6.0930e-01,
         9.0591e-01,  1.7857e-01])


Words are now embedded using a concatenation of three different embeddings. This means that the resulting embedding vector is still a single PyTorch vector.

## Document Embeddings

Document embeddings are different from word embeddings in that they provide one embedding for an entire text, whereas word embeddings provide embeddings for individual words.

...

**(C) 2018-2020 by [Damir Cavar](http://damir.cavar.me/)**